In [5]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import pandas as pd
import time


## 함수 -> 제목,링크,날짜를 뽑음

In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime, timedelta

# 속보랑 단독 들어가 있는 경우만 제목과 링크를 크롤링하는 함수 
def get_special_news_titles_and_links(url):
    response = requests.get(url)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')
    titles_and_links = [(a.get_text().strip(), a['href']) for a in soup.select('.rankingnews_box .rankingnews_list a') if a.get_text().strip() and a.get_text().strip() != '동영상기사']

    titles=[title for title, link in titles_and_links]
    links = [link for title, link in titles_and_links]
    return titles, links

# 날짜 생성 함수
def generate_dates(start_date, end_date):
    start = datetime.strptime(start_date, '%Y%m%d')
    end = datetime.strptime(end_date, '%Y%m%d')
    delta = timedelta(days=1)
    current = start
    while current <= end:
        yield current.strftime('%Y%m%d')
        current += delta


## 댓글 많은 뉴스+ 많이 본 뉴스

In [8]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime, timedelta

def generate_dates(start_date, end_date):
    start = datetime.strptime(start_date, '%Y%m%d')
    end = datetime.strptime(end_date, '%Y%m%d')
    delta = timedelta(days=1)
    current = start
    while current <= end:
        yield current.strftime('%Y%m%d')
        current += delta

def get_special_news_titles_and_links(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    titles = [tag.text.strip() for tag in soup.select('.list_title')]
    links = [tag['href'] for tag in soup.select('.list_title')]
    return titles, links

base_urls = [
    "https://news.naver.com/main/ranking/popularDay.naver?date=",
    "https://news.naver.com/main/ranking/popularMemo.naver?date="
]

start_date = '20230701'
end_date = '20240701'

for date in generate_dates(start_date, end_date):
    all_articles = []
    for base_url in base_urls:
        url = base_url + date
        titles, links = get_special_news_titles_and_links(url)
        for title, link in zip(titles, links):
            all_articles.append({
                'Date': date,
                'Title': title,
                'Link': link
            })

    df = pd.DataFrame(all_articles)
    df.to_csv(f'뉴스//important_news_{date}.csv', index=False)


In [4]:
df.shape

(827, 3)